In [1]:
#!/usr/bin/env python

import numpy

dataDir='C:/Users/navne/Python Files/Data Acquisition BIA 627/Final Project/'
file_name_test=dataDir + 'testTrack_hierarchy.txt'
file_name_train=dataDir + 'trainIdx2_matrix.txt'
output_file= dataDir + 'output1.txt'

fTest= open(file_name_test, 'r')
fTrain=open(file_name_train, 'r')
Trainline= fTrain.readline()
fOut = open(output_file, 'w')

trackID_vec=[0]*6
albumID_vec=[0]*6
artistID_vec=[0]*6
lastUserID=-1

user_rating_inTrain=numpy.zeros(shape=(6,3))

for line in fTest:
	arr_test=line.strip().split('|')
	userID= arr_test[0]
	trackID= arr_test[1]
	albumID= arr_test[2]
	artistID=arr_test[3]

	if userID!= lastUserID:
		ii=0
		user_rating_inTrain=numpy.zeros(shape=(6,3))

	trackID_vec[ii]=trackID
	albumID_vec[ii]=albumID
	artistID_vec[ii]=artistID
	ii=ii+1
	lastUserID=userID

	if ii==6 : 
		while (Trainline):
		# for Trainline in fTrain:
			arr_train = Trainline.strip().split('|')
			trainUserID=arr_train[0]
			trainItemID=arr_train[1]
			trainRating=arr_train[2]
			Trainline=fTrain.readline()		

			if trainUserID< userID:
				continue
			if trainUserID== userID:				
				for nn in range(0, 6):
					if trainItemID==albumID_vec[nn]:
						user_rating_inTrain[nn, 0]=trainRating
					if trainItemID==artistID_vec[nn]:
						user_rating_inTrain[nn, 1]=trainRating
			if trainUserID> userID:
				for nn in range(0, 6):
					outStr=str(userID) + '|' + str(trackID_vec[nn])+ '|' + str(user_rating_inTrain[nn,0]) + '|' + str(user_rating_inTrain[nn, 1])
					fOut.write(outStr + '\n')
				break



fTest.close()
fTrain.close()





In [3]:
import numpy as np

# Set up file paths
dataDir = 'C:/Users/navne/Python Files/Data Acquisition BIA 627/Final Project/'
file_name_test = dataDir + 'testTrack_hierarchy.txt'
file_name_train = dataDir + 'trainIdx2_matrix.txt'
output_file = dataDir + 'output1.txt'

# Load training data into a dictionary
train_data = {}
with open(file_name_train, 'r') as fTrain:
    for line in fTrain:
        userID, itemID, rating = line.strip().split('|')[:3]
        if userID not in train_data:
            train_data[userID] = {}
        # Store ratings by album and artist ID
        train_data[userID][itemID] = int(rating)

# Process test data and write predictions
with open(file_name_test, 'r') as fTest, open(output_file, 'w') as fOut:
    lastUserID = None  # Track the last user ID processed
    trackID_vec = []  # Initialize the track ID vector for each user

    for line in fTest:
        arr_test = line.strip().split('|')
        if len(arr_test) < 4:
            continue  # Skip if line doesn't have enough data
        userID, trackID, albumID, artistID = arr_test[:4]

        # Reset trackID_vec for a new user
        if userID != lastUserID:
            if lastUserID is not None:
                # Write predictions for the previous user's tracks
                for i, predRating in enumerate(predRatings):
                    outStr = f"{lastUserID}|{trackID_vec[i]}|{int(predRating > 0)}\n"  # Convert to 0 or 1 based on whether a rating exists
                    fOut.write(outStr)

            trackID_vec = [trackID]  # Start a new list for the new user
            predRatings = np.zeros(6)  # Reset predicted ratings for the new user
        else:
            trackID_vec.append(trackID)  # Append trackID for the same user

        # Check if the user exists in the training data
        if userID in train_data:
            user_ratings = train_data[userID]

            # Predict based on album or artist rating, if available
            if albumID in user_ratings:
                predRatings[len(trackID_vec) - 1] = user_ratings[albumID]
            elif artistID in user_ratings:
                predRatings[len(trackID_vec) - 1] = user_ratings[artistID]

        lastUserID = userID  # Update the last user ID processed

    # Write predictions for the last user in the file
    for i, predRating in enumerate(predRatings):
        outStr = f"{userID}|{trackID_vec[i]}|{int(predRating > 0)}\n"
        fOut.write(outStr)


In [4]:
import numpy as np
import csv

# Set up file paths
dataDir = 'C:/Users/navne/Python Files/Data Acquisition BIA 627/Final Project/'
file_name_test = dataDir + 'testTrack_hierarchy.txt'
file_name_train = dataDir + 'trainIdx2_matrix.txt'
output_file = dataDir + 'output1.csv'  # Changed the extension to .csv

# Loading the training data into a dictionary
train_data = {}
with open(file_name_train, 'r') as fTrain:
    for line in fTrain:
        userID, itemID, rating = line.strip().split('|')[:3]
        if userID not in train_data:
            train_data[userID] = {}
        train_data[userID][itemID] = int(rating)

# then we process the test data and write predictions
with open(file_name_test, 'r') as fTest, open(output_file, 'w', newline='') as fOut:
    csv_writer = csv.writer(fOut)
    csv_writer.writerow(['TrackID', 'Predictor'])  # Into the csv file we will add the header

    lastUserID = None  # Track the last user ID processed
    trackID_vec = []  # Initialize the track ID vector for each user

    for line in fTest:
        arr_test = line.strip().split('|')
        if len(arr_test) < 4:
            continue  # we will skip if  then line doesn't have enough data
        userID, trackID, albumID, artistID = arr_test[:4]   # only considering these 4 entities 

        # Reset trackID_vec for a new user
        if userID != lastUserID:
            if lastUserID is not None:
                # Write predictions for the previous user's tracks
                for i, predRating in enumerate(predRatings):
                    trackID_pred = f"{lastUserID}_{trackID_vec[i]}"
                    csv_writer.writerow([trackID_pred, int(predRating > 0)]) 

            trackID_vec = [trackID]  # Start a new list for the new user
            predRatings = np.zeros(6)  # Reset predicted ratings for the new user
        else:
            trackID_vec.append(trackID)  # Append trackID for the same user

        # Check if the user exists in the training data
        if userID in train_data:
            user_ratings = train_data[userID]

            # Predict based on album or artist rating, if available
            if albumID in user_ratings:
                predRatings[len(trackID_vec) - 1] = user_ratings[albumID]
            elif artistID in user_ratings:
                predRatings[len(trackID_vec) - 1] = user_ratings[artistID]

        lastUserID = userID  # Update the last user ID processed

    # Write predictions for the last user in the file
    for i, predRating in enumerate(predRatings):
        trackID_pred = f"{userID}_{trackID_vec[i]}"
        csv_writer.writerow([trackID_pred, int(predRating > 0)])  # Write to CSV file


In [3]:
import numpy as np
import csv
from scipy.stats import pearsonr
from collections import defaultdict

# Set up file paths
dataDir = 'C:/Users/navne/Python Files/Data Acquisition BIA 627/Final Project/'
file_name_test = dataDir + 'testTrack_hierarchy.txt'
file_name_train = dataDir + 'trainIdx2_matrix.txt'
output_file = dataDir + 'output1_2.csv'

# Load training data
train_data = defaultdict(dict)
with open(file_name_train, 'r') as fTrain:
    for line in fTrain:
        userID, itemID, rating = line.strip().split('|')[:3]
        train_data[userID][itemID] = int(rating)

# Compute user similarities
def compute_similarities(train_data):
    similarities = {}
    users = list(train_data.keys())
    for i, user1 in enumerate(users):
        for user2 in users[i+1:]:
            # Compute Pearson correlation coefficient between user1 and user2
            common_items = set(train_data[user1]).intersection(set(train_data[user2]))
            if len(common_items) > 0:
                ratings1 = [train_data[user1][item] for item in common_items]
                ratings2 = [train_data[user2][item] for item in common_items]
                similarity, _ = pearsonr(ratings1, ratings2)
                similarities[(user1, user2)] = similarity
    return similarities

# Predict rating
def predict_rating(user, item, train_data, similarities, k=10):
    # Find k most similar users who have rated this item
    scores = []
    for (user1, user2), similarity in similarities.items():
        other_user = user2 if user1 == user else user1
        if item in train_data[other_user]:
            scores.append((similarity, train_data[other_user][item]))
    scores.sort(reverse=True)
    scores = scores[:k]  # Keep top k scores
    
    # Compute weighted average
    if scores:
        weighted_sum = sum(similarity * rating for similarity, rating in scores)
        sum_of_weights = sum(abs(similarity) for similarity, _ in scores)
        return weighted_sum / sum_of_weights if sum_of_weights > 0 else 0
    return 0  # Fallback if no similar users have rated this item

# Precompute similarities
similarities = compute_similarities(train_data)

# Process test data and make predictions
with open(file_name_test, 'r') as fTest, open(output_file, 'w', newline='') as fOut:
    csv_writer = csv.writer(fOut)
    csv_writer.writerow(['TrackID', 'Predictor'])

    for line in fTest:
        userID, trackID, _, _ = line.strip().split('|')[:4]
        predRating = predict_rating(userID, trackID, train_data, similarities)
        csv_writer.writerow([f"{userID}_{trackID}", int(predRating >= 0.5)])


C:\Users\navne\AppData\Roaming\Python\Python39\site-packages\scipy\stats\stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


ValueError: x and y must have length at least 2.